In [1]:
%pylab nbagg

Populating the interactive namespace from numpy and matplotlib


In [2]:
#!/bin/python3

from physdata.xray import *  # pip install physdata
import numpy as np

element_density = []

for element in fetch_elements():
    
    element_density.append(element.density)

In [3]:
import re

names = []
weights = []
elements = []
density = []

element_number = []
# element_density = []

with open('../data/geant4_materials.txt','r') as f:
    
    lines = f.readlines()
        
    for ii in range(len(lines)):
        
        line = lines[ii]
        
        if ' AddMaterial' in line:
            
            seps = line.split('"')
            
            if len(seps) > 1:
                
                if len(seps[1]) > 5:
                    
                    jj = ii + 1
                    
                    if ' AddElementByAtom' in lines[jj]:
                        continue
                    
                    weight = []
                    element = []
                    
                    while ' AddElementByWeightFraction' in lines[jj]:
                        
                        numbers = re.split('[\(\)]',lines[jj])[1].split(',')
                        
                        element.append(int(numbers[0]))
                        weight.append(float(numbers[1]))
                        
                        jj += 1
                    
                    elements.append(element)
                    weights.append(weight)    
                    names.append(seps[1])
                    
                    density.append(float(seps[2].split(',')[1]))
                    
                else:
                    element_number.append(int(seps[2].split(',')[2]))
#                     element_density.append(float(seps[2].split(',')[1]))

In [4]:
H = np.loadtxt('/home/xcite/fastcat-0.0.1/fastcat/data/mu/1.csv',delimiter=',')

import fastcat.fastcat as fc

for kk, name in enumerate(names):
    
    attenuations = []
    energies = []
    lens = []
    
    for weight, element in zip(weights[kk],elements[kk]):
        
        attenuation = np.loadtxt('/home/xcite/fastcat-0.0.1/fastcat/data/mu/'+str(element)+'.csv',delimiter=',')[1]
        attenuation = attenuation/element_density[element-1]*weight
        
#         print(element_density[element-1])
        
        attenuations.append(attenuation)
        energies.append(np.loadtxt('/home/xcite/fastcat-0.0.1/fastcat/data/mu/'+str(element)+'.csv',delimiter=',')[0])
        lens.append(len(attenuation))
        
    if max(lens) == 36:
        
        attenuation_all = np.zeros([2,len(H[0])])
        attenuation_all[0] = H[0]
        
        for ii in range(len(attenuations)):
            
            attenuation_all[1] += attenuations[ii]
        
    else:
        
        energies_larger = np.unique(np.hstack(energies))
        
        attenuation_all = np.zeros([2,len(energies_larger)])
        attenuation_all[0] = energies_larger
        
        for ii in range(len(attenuations)):
            
            f = fc.log_interp_1d(energies[ii],attenuations[ii])
            
            attenuation_all[1] += f(energies_larger)
            
        
    attenuation_all[1] *= density[kk]
    np.savetxt('../elements/'+name+".csv",attenuation_all, fmt="%.8G",delimiter=",")
    
#     if name == 'G4_TEFLON':
#         break

In [5]:
!mv ../elements/* ../../fastcat/data/mu/

In [197]:
np.loadtxt('/home/xcite/xpecgen/xpecgen/data/mu/'+'9'+'.csv',delimiter=',')[1]/element_density[8]

array([5.649e+03, 1.979e+03, 9.047e+02, 2.888e+02, 1.256e+02, 6.514e+01,
       3.789e+01, 1.602e+01, 8.205e+00, 2.492e+00, 1.133e+00, 4.487e-01,
       2.828e-01, 2.214e-01, 1.920e-01, 1.639e-01, 1.496e-01, 1.298e-01,
       1.176e-01, 1.015e-01, 9.073e-02, 8.274e-02, 7.649e-02, 6.717e-02,
       6.037e-02, 5.399e-02, 4.915e-02, 4.228e-02, 3.422e-02, 2.960e-02,
       2.663e-02, 2.457e-02, 2.195e-02, 2.039e-02, 1.846e-02, 1.769e-02])

In [202]:
(np.loadtxt('/home/xcite/xpecgen/xpecgen/data/mu/'+'9'+'.csv',delimiter=',')[1]/element_density[8]*.76 +
 np.loadtxt('/home/xcite/xpecgen/xpecgen/data/mu/'+'6'+'.csv',delimiter=',')[1]/element_density[5]*.24)*density[kk]

array([1.0612536e+04, 3.6785936e+03, 1.6724312e+03, 5.3056784e+02,
       2.2995104e+02, 1.1900944e+02, 6.9133680e+01, 2.9201568e+01,
       1.4971704e+01, 4.5927728e+00, 2.1277520e+00, 8.8550000e-01,
       5.8245440e-01, 4.6896960e-01, 4.1358240e-01, 3.5904880e-01,
       3.3007040e-01, 2.8814720e-01, 2.6151840e-01, 2.2599280e-01,
       2.0210344e-01, 1.8435648e-01, 1.7043752e-01, 1.4966952e-01,
       1.3452472e-01, 1.2031448e-01, 1.0952392e-01, 9.4145920e-02,
       7.6023200e-02, 6.5579360e-02, 5.8823600e-02, 5.4117360e-02,
       4.8073520e-02, 4.4435600e-02, 3.9830560e-02, 3.7893680e-02])

In [18]:
element_density[16]

0.002995

In [188]:
weights[kk][0]

0.240183

In [75]:
physdata.xray.ElementData()

In [67]:
H[0].shape

(36,)

In [69]:
elements[kk]

[1, 6, 7, 8, 9, 20]